## example of fitting an NN in TF, just changed a couple of items.


In [12]:
from __future__ import print_function

import math

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.python.data import Dataset
# USE SHIFT+TAB FOR DOC STRING IN FUNCTION
tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

In [13]:
housing_df = pd.read_csv('train_LR.csv', sep = ',')
housing_df = shuffle(housing_df) #make sure not pre-ordered.

In [14]:
housing_df;

In [15]:
#breakup into features and labels (targets)
processed_features = housing_df[['GrLivArea']]
output_targets = housing_df[['SalePrice']]

In [16]:
#SPLIT INTO 3 PARTS:  TRAIN, VAL, TEST
training_examples = processed_features[0:1060]
training_targets = output_targets[0:1060]

val_examples = processed_features[1060:1260]
val_targets = output_targets[1060:1260]

test_examples = processed_features[1260:1460]
test_targets = output_targets[1260:1460]

In [17]:
# build model

# CONFIGURE A NUMERIC FEATURE COLUMN FOR GRLIVAREA
my_feature_columns = [tf.feature_column.numeric_column('GrLivArea')]

# Define the preferred optimizer:  in this case lets use gradient descent
#my_optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01) #too small, big error, now chnage on second run to 0.1
my_optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)

# configure the LR model without feature columns and optimizer:
#model = tf.estimator.LinearRegressor(feature_columns=my_feature_columns, optimizer=my_optimizer)
model = tf.estimator.DNNRegressor(feature_columns=my_feature_columns, hidden_units=[12,12], optimizer=my_optimizer)

In [18]:
# Define the input function:
def my_input_fn(features, targets, batch_size=1, shuffle=True, num_epochs=None):
    # Convert pandas data ina a dict of np arrays
    features = {key:np.array(value) for key, value in dict(features).items()}

    # Construct a datasetj, and configure batching/reapeating.
    ds = Dataset.from_tensor_slices((features, targets)) # warning:  2GB limit
    ds = ds.batch(batch_size).repeat(num_epochs)

    # Shuffle the data, if specified
    if shuffle:
        ds = ds.shuffle(buffer_size=10000)

    # Return the next batch of data
    features, labels = ds.make_one_shot_iterator().get_next()

    return features, labels

In [19]:
# Train the model from existing data
training = model.train(input_fn = lambda:my_input_fn(training_examples, training_targets), steps=1000)

In [20]:
# Evaluate the model with RMSE:
train_predictions = model.predict(input_fn = lambda: my_input_fn(training_examples, training_targets, num_epochs=1, shuffle=False))
val_predictions = model.predict(input_fn = lambda: my_input_fn(val_examples, val_targets, num_epochs=1, shuffle=False))
test_predictions = model.predict(input_fn = lambda: my_input_fn(test_examples, test_targets, num_epochs=1, shuffle=False))

# Format predictions as np arrays so we can calc error metrics
train_predictions = np.array([item['predictions'][0] for item in train_predictions])
val_predictions = np.array([item['predictions'][0] for item in val_predictions])
test_predictions = np.array([item['predictions'][0] for item in test_predictions])

# print MSE and RMSE:
mean_squared_error = metrics.mean_squared_error(train_predictions, training_targets)
root_mean_squared_error = math.sqrt(mean_squared_error)
print(f'Root mean squared error (on training data): {root_mean_squared_error}')
mean_squared_error = metrics.mean_squared_error(val_predictions, val_targets)
root_mean_squared_error = math.sqrt(mean_squared_error)
print(f'Root mean squared error (on val data): {root_mean_squared_error}')
mean_squared_error = metrics.mean_squared_error(test_predictions, test_targets)
root_mean_squared_error = math.sqrt(mean_squared_error)
print(f'Root mean squared error (on test data): {root_mean_squared_error}')

Root mean squared error (on training data): 71811.6214101775
Root mean squared error (on val data): 85811.78925133696
Root mean squared error (on test data): 59984.19457899401


In [21]:
"""# Evaluate the model with visuals:
train_sample = housing_df[0:1060].sample(30)
val_sample = housing_df[1060:1260].sample(10)
test_sample = housing_df[1260:1460].sample(10)

# get the min and max total_rooms values
x_0 = train_sample['GrLivArea'].min()
x_1 = train_sample['GrLivArea'].max()

# retrieve the final weight and bias generated while training:
weight = model.get_variable_value('linear/linear_model/GrLivArea/weights')[0]
bias = model.get_variable_value('linear/linear_model/bias_weights')

# get the predicted medeian_house_values for the min and max total_rooms values
y_0 = weight * x_0 + bias
y_1 = weight * x_1 + bias

# plot our regression line from (x_0, y_0) to (x_1, y_1):
plt.plot([x_0, x_1], [y_0, y_1], c='r')

# label the graph axes
plt.ylabel('SalePrice')
plt.xlabel('GrLivArea')

# plot a scatter plot from our data sample
plt.scatter(train_sample['GrLivArea'], train_sample['SalePrice'])
plt.scatter(val_sample['GrLivArea'], val_sample['SalePrice'])
plt.scatter(test_sample['GrLivArea'], test_sample['SalePrice'])

# display the graph
plt.show()"""

"# Evaluate the model with visuals:\ntrain_sample = housing_df[0:1060].sample(30)\nval_sample = housing_df[1060:1260].sample(10)\ntest_sample = housing_df[1260:1460].sample(10)\n\n# get the min and max total_rooms values\nx_0 = train_sample['GrLivArea'].min()\nx_1 = train_sample['GrLivArea'].max()\n\n# retrieve the final weight and bias generated while training:\nweight = model.get_variable_value('linear/linear_model/GrLivArea/weights')[0]\nbias = model.get_variable_value('linear/linear_model/bias_weights')\n\n# get the predicted medeian_house_values for the min and max total_rooms values\ny_0 = weight * x_0 + bias\ny_1 = weight * x_1 + bias\n\n# plot our regression line from (x_0, y_0) to (x_1, y_1):\nplt.plot([x_0, x_1], [y_0, y_1], c='r')\n\n# label the graph axes\nplt.ylabel('SalePrice')\nplt.xlabel('GrLivArea')\n\n# plot a scatter plot from our data sample\nplt.scatter(train_sample['GrLivArea'], train_sample['SalePrice'])\nplt.scatter(val_sample['GrLivArea'], val_sample['SalePrice